In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Getting Data

In [0]:
train=pd.read_csv("/content/drive/My Drive/My Colab/AmExpert2019/train_AUpWtIz/train.csv")
item_data=pd.read_csv("/content/drive/My Drive/My Colab/AmExpert2019/train_AUpWtIz/item_data.csv")
customer_transaction_data=pd.read_csv("/content/drive/My Drive/My Colab/AmExpert2019/train_AUpWtIz/customer_transaction_data.csv")
customer_demographics=pd.read_csv("/content/drive/My Drive/My Colab/AmExpert2019/train_AUpWtIz/customer_demographics.csv")
coupon_item_mapping=pd.read_csv("/content/drive/My Drive/My Colab/AmExpert2019/train_AUpWtIz/coupon_item_mapping.csv")
campaign_data=pd.read_csv("/content/drive/My Drive/My Colab/AmExpert2019/train_AUpWtIz/campaign_data.csv")
test=pd.read_csv("/content/drive/My Drive/My Colab/AmExpert2019/test_QyjYwdj.csv")

In [0]:
test.head()

,id,campaign_id,coupon_id,customer_id
0,3,22,869,967
1,4,20,389,1566
2,5,22,981,510
3,8,25,1069,361
4,10,17,498,811


In [0]:
train["redemption_status"].value_counts()

0    77640
1      729
Name: redemption_status, dtype: int64

In [0]:
print("Train",train.shape)
print("Test",test.shape)

Train (78369, 5)
Test (50226, 4)


In [0]:
rs= pd.DataFrame(train['redemption_status'])
train.drop(['redemption_status'], axis=1, inplace=True)

In [0]:
rs.head()

,redemption_status
0,0
1,0
2,0
3,0
4,0


In [0]:
rs.shape

(78369, 1)

In [0]:
train['Type']='Train' #Create a flag for Train and Test Data set
test['Type']='Test'
combi= pd.concat([train,test],axis=0) #Combined both Train and Test Data set

In [0]:
combi.head()

,id,campaign_id,coupon_id,customer_id,Type
0,1,13,27,1053,Train
1,2,13,116,48,Train
2,6,9,635,205,Train
3,7,13,644,1050,Train
4,9,8,1017,1489,Train


In [0]:
import featuretools as ft
# Create new entityset
es = ft.EntitySet(id = 'coupon_train')

# Create an entity from the client dataframe
# This dataframe already has an index and a time index
es = es.entity_from_dataframe(entity_id = 'combi', dataframe = combi,index = 'id')
es = es.entity_from_dataframe(entity_id = 'campaign_data',dataframe = campaign_data,index = 'campaign_id')
es = es.entity_from_dataframe(entity_id = 'customer_demographics',dataframe = customer_demographics,index = 'customer_id')
es = es.entity_from_dataframe(entity_id = 'item_data',dataframe = item_data,index = 'item_id')
es = es.entity_from_dataframe(entity_id = 'coupon_item_mapping', 
                              dataframe = coupon_item_mapping,
                              index="coupon_item_mapping_id",
                              make_index = True)
es = es.entity_from_dataframe(entity_id = 'customer_transaction_data', 
                              dataframe = customer_transaction_data,
                              index="customer_transaction_data_id",
                              make_index = True)

# Relationship between train and previous campaign_data
r1 = ft.Relationship(es['campaign_data']['campaign_id'],es['combi']['campaign_id'])
# Add the relationship to the entity set
es = es.add_relationship(r1)

# Relationship between previous loans and previous payments
r2 = ft.Relationship(es['customer_demographics']['customer_id'],es['combi']['campaign_id'])
# Add the relationship to the entity set
es = es.add_relationship(r2)

#r3 = ft.Relationship(es['coupon_item_mapping']['coupon_id'],es['combi']['coupon_id'])
# Add the relationship to the entity set
#es = es.add_relationship(r3)

r4 = ft.Relationship(es['item_data']['item_id'], es['coupon_item_mapping']['item_id'])
# Add the relationship to the entity set
es = es.add_relationship(r4)

r5 = ft.Relationship(es['item_data']['item_id'], es['customer_transaction_data']['item_id'])
# Add the relationship to the entity set
es = es.add_relationship(r5)

r6 = ft.Relationship(es['customer_demographics']['customer_id'], es['customer_transaction_data']['customer_id'])
# Add the relationship to the entity set
es = es.add_relationship(r6)

#r7 = ft.Relationship(es['customer_transaction_data']['customer_id'],es['combi']['customer_id'])
# Add the relationship to the entity set
#es = es.add_relationship(r7)

# Create new features using specified primitives
#combi, feature_names = ft.dfs(entityset = es, target_entity = 'combi', 
#                                 agg_primitives = ['mean', 'max', 'percent_true', 'last'],
 #                                trans_primitives = ['year', 'month','day','week'])
#

combi, feature_names = ft.dfs(entityset=es, 
target_entity = 'combi')

In [0]:
combi.shape

(128595, 76)

In [0]:
cols = combi.columns
num_cols = combi._get_numeric_data().columns
list(set(cols) - set(num_cols))

['Type',
 'campaign_data.MODE(combi.Type)',
 'customer_demographics.no_of_children',
 'campaign_data.campaign_type',
 'customer_demographics.age_range',
 'customer_demographics.MODE(combi.Type)',
 'customer_demographics.family_size',
 'customer_demographics.marital_status']

In [0]:
combi.head()

,Type,campaign_id,coupon_id,customer_id,redemption_status,campaign_data.campaign_type,customer_demographics.age_range,customer_demographics.marital_status,customer_demographics.rented,customer_demographics.family_size,customer_demographics.no_of_children,customer_demographics.income_bracket,campaign_data.SUM(combi.coupon_id),campaign_data.SUM(combi.customer_id),campaign_data.SUM(combi.redemption_status),campaign_data.STD(combi.coupon_id),campaign_data.STD(combi.customer_id),campaign_data.STD(combi.redemption_status),campaign_data.MAX(combi.coupon_id),campaign_data.MAX(combi.customer_id),campaign_data.MAX(combi.redemption_status),campaign_data.SKEW(combi.coupon_id),campaign_data.SKEW(combi.customer_id),campaign_data.SKEW(combi.redemption_status),campaign_data.MIN(combi.coupon_id),campaign_data.MIN(combi.customer_id),campaign_data.MIN(combi.redemption_status),campaign_data.MEAN(combi.coupon_id),campaign_data.MEAN(combi.customer_id),campaign_data.MEAN(combi.redemption_status),campaign_data.COUNT(combi),campaign_data.NUM_UNIQUE(combi.Type),campaign_data.MODE(combi.Type),campaign_data.DAY(start_date),campaign_data.DAY(end_date),campaign_data.YEAR(start_date),campaign_data.YEAR(end_date),campaign_data.MONTH(start_date),campaign_data.MONTH(end_date),campaign_data.WEEKDAY(start_date),...,customer_demographics.MAX(combi.redemption_status),customer_demographics.SKEW(combi.coupon_id),customer_demographics.SKEW(combi.customer_id),customer_demographics.SKEW(combi.redemption_status),customer_demographics.MIN(combi.coupon_id),customer_demographics.MIN(combi.customer_id),customer_demographics.MIN(combi.redemption_status),customer_demographics.MEAN(combi.coupon_id),customer_demographics.MEAN(combi.customer_id),customer_demographics.MEAN(combi.redemption_status),customer_demographics.COUNT(combi),customer_demographics.NUM_UNIQUE(combi.Type),customer_demographics.MODE(combi.Type),customer_demographics.SUM(customer_transaction_data.quantity),customer_demographics.SUM(customer_transaction_data.selling_price),customer_demographics.SUM(customer_transaction_data.other_discount),customer_demographics.SUM(customer_transaction_data.coupon_discount),customer_demographics.STD(customer_transaction_data.quantity),customer_demographics.STD(customer_transaction_data.selling_price),customer_demographics.STD(customer_transaction_data.other_discount),customer_demographics.STD(customer_transaction_data.coupon_discount),customer_demographics.MAX(customer_transaction_data.quantity),customer_demographics.MAX(customer_transaction_data.selling_price),customer_demographics.MAX(customer_transaction_data.other_discount),customer_demographics.MAX(customer_transaction_data.coupon_discount),customer_demographics.SKEW(customer_transaction_data.quantity),customer_demographics.SKEW(customer_transaction_data.selling_price),customer_demographics.SKEW(customer_transaction_data.other_discount),customer_demographics.SKEW(customer_transaction_data.coupon_discount),customer_demographics.MIN(customer_transaction_data.quantity),customer_demographics.MIN(customer_transaction_data.selling_price),customer_demographics.MIN(customer_transaction_data.other_discount),customer_demographics.MIN(customer_transaction_data.coupon_discount),customer_demographics.MEAN(customer_transaction_data.quantity),customer_demographics.MEAN(customer_transaction_data.selling_price),customer_demographics.MEAN(customer_transaction_data.other_discount),customer_demographics.MEAN(customer_transaction_data.coupon_discount),customer_demographics.COUNT(customer_transaction_data),customer_demographics.NUM_UNIQUE(customer_transaction_data.item_id),customer_demographics.MODE(customer_transaction_data.item_id)
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Train,13,27,1053,0.0,X,36-45,Single,0.0,1,NaN,2.0,11436200,17700224,346.0,337.453434,453.812706,0.122768,1114,1582,1.0,0.198288,0.031854,7.896777,20,1,0.0,505.892241,782.987879,0.015306,22606,1,Train,19,7,2013,2013,5,5,6,...,1.0,0.

In [0]:
combi.drop(['customer_demographics.age_range'], axis=1, inplace=True)
combi.drop(['campaign_data.campaign_type'], axis=1, inplace=True)
combi.drop(['customer_demographics.family_size'], axis=1, inplace=True)
combi.drop(['customer_demographics.no_of_children'], axis=1, inplace=True)
combi.drop(['customer_demographics.marital_status'], axis=1, inplace=True)

In [0]:
#labelencoder = LabelEncoder()
#combi["customer_demographics.age_range"] = labelencoder.fit_transform(combi["customer_demographics.age_range"])
#combi["campaign_data.campaign_type"] = labelencoder.fit_transform(combi["campaign_data.campaign_type"])
#combi["customer_demographics.family_size"] = labelencoder.fit_transform(combi["customer_demographics.family_size"])
#combi["customer_demographics.no_of_children"] = labelencoder.fit_transform(combi["customer_demographics.no_of_children"])
#combi["customer_demographics.marital_status"] = labelencoder.fit_transform(combi["customer_demographics.marital_status"])

In [0]:
ntrain=combi[combi['Type']=='Train']
ntest=combi[combi['Type']=='Test']

In [0]:
ntrain.drop(['Type'], axis=1, inplace=True)
ntest.drop(['Type'], axis=1, inplace=True)
ntrain.drop(['campaign_data.MODE(combi.Type)'], axis=1, inplace=True)
ntest.drop(['campaign_data.MODE(combi.Type)'], axis=1, inplace=True)
ntrain.drop(['customer_demographics.MODE(combi.Type)'], axis=1, inplace=True)
ntest.drop(['customer_demographics.MODE(combi.Type)'], axis=1, inplace=True)
#ntrain.drop(['customer_demographics.age_range'], axis=1, inplace=True)
#ntest.drop(['customer_demographics.age_range'], axis=1, inplace=True)

In [0]:
ntest.shape

(50226, 68)

In [0]:
ntrain=ntrain.fillna(ntrain.mean())
ntest=ntest.fillna(ntest.mean())

In [0]:
ntest.shape

(50226, 68)

In [0]:
X=ntrain
y=rs

In [0]:
#X.to_csv("X.csv")
#rs.to_csv("y.csv")
#ntest.to_csv("test.csv")

In [0]:
X=X.reset_index()
ntest=ntest.reset_index()
X.drop(['id'], axis=1, inplace=True)

In [0]:
#newtrain=pd.concat([X,y],axis=1)
#newtrain.to_csv("train.csv")

(78369, 69)

In [0]:
#xcol=X.columns
#ycol=y.columns

In [0]:
X=X.values
y=y.values

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [0]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_train, y_train= ros.fit_resample(X_train, y_train)

In [0]:
X_train=pd.DataFrame(X_train)
y_train=pd.DataFrame(y_train)
X_test=pd.DataFrame(X_test)
y_test=pd.DataFrame(y_test)
X_train.columns=xcol
X_test.columns=xcol
y_train.columns=ycol
y_test.columns=ycol

In [0]:
#from imblearn.over_sampling import SMOTE
#sm = SMOTE(random_state=2)
#X_train, y_train = sm.fit_sample(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)
y_pred=knn.predict(X_test)
from sklearn import metrics
y_pred=knn.predict(X_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test,y_pred, pos_label=1)
print("AUC: ",metrics.auc(fpr, tpr))

AUC:  0.5369682134559222


In [0]:
#RF
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
from sklearn import metrics
y_pred=rf.predict(X_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test,y_pred, pos_label=1)
print("AUC: ",metrics.auc(fpr, tpr))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


AUC:  0.5384237700065175


In [0]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [0]:
#BME
from sklearn.ensemble import BaggingClassifier
from sklearn import tree
bme= BaggingClassifier(tree.DecisionTreeClassifier(random_state=1))
bme.fit(X_train,y_train)
y_pred=bme.predict(X_test)
from sklearn import metrics
y_pred=bme.predict(X_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test,y_pred, pos_label=1)
print("AUC: ",metrics.auc(fpr, tpr))

AUC:  0.5577611233003478


In [0]:
#ADABOOST
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
ada1 = AdaBoostClassifier()
param_grid = {}
ada = GridSearchCV(ada1, param_grid, cv=2)
ada.fit(X_train, y_train)
from sklearn import metrics
y_pred=ada.predict(X_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test,y_pred, pos_label=1)
print("AUC: ",metrics.auc(fpr, tpr))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AUC:  0.6783736200042787


In [0]:
#GBM
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
gbm1 = GradientBoostingClassifier()
param_grid = {}
gbm_gscv = GridSearchCV(gbm1, param_grid, cv=2)
gbm_gscv.fit(X_train, y_train)
from sklearn import metrics
y_pred=gbm_gscv.predict(X_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test,y_pred, pos_label=1)
print("AUC: ",metrics.auc(fpr, tpr))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AUC:  0.503448275862069


In [0]:
#XGBOOST
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
xgb1=XGBClassifier()
param_grid = {}
xgb_gscv = GridSearchCV(xgb1, param_grid, cv=7)
xgb_gscv.fit(X_train, y_train)
from sklearn import metrics
y_pred=xgb_gscv.predict(X_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test,y_pred, pos_label=1)
print("AUC: ",metrics.auc(fpr, tpr))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AUC:  0.7358825629238248


In [0]:
#LGBM
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
lgbm1=LGBMClassifier()
param_grid = {}
lgbm_gscv = GridSearchCV(lgbm1, param_grid, cv=3)
lgbm_gscv.fit(X_train, y_train)
from sklearn import metrics
y_pred=lgbm_gscv.predict(X_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test,y_pred, pos_label=1)
print("AUC: ",metrics.auc(fpr, tpr))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AUC:  0.7329030408414056


In [0]:
pip install catboost

     |████████████████████████████████| 62.6MB 438kB/s 


In [0]:
#CATBOOST
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
cat=CatBoostClassifier()
param_grid = {"n_estimators":[100]}
cat_gscv = GridSearchCV(cat, param_grid, cv=3)
cat_gscv.fit(X_train, y_train)
from sklearn import metrics
y_pred=cat_gscv.predict(X_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test,y_pred, pos_label=1)
print("AUC: ",metrics.auc(fpr, tpr))

Learning rate set to 0.438038
0:	learn: 0.5774593	total: 95.2ms	remaining: 9.42s
1:	learn: 0.5520627	total: 136ms	remaining: 6.68s
2:	learn: 0.5339520	total: 177ms	remaining: 5.73s
3:	learn: 0.5172096	total: 220ms	remaining: 5.28s
4:	learn: 0.5073728	total: 260ms	remaining: 4.93s
5:	learn: 0.5000358	total: 298ms	remaining: 4.67s
6:	learn: 0.4939681	total: 337ms	remaining: 4.48s
7:	learn: 0.4902338	total: 377ms	remaining: 4.33s
8:	learn: 0.4876881	total: 413ms	remaining: 4.18s
9:	learn: 0.4838110	total: 458ms	remaining: 4.12s
10:	learn: 0.4780464	total: 498ms	remaining: 4.03s
11:	learn: 0.4747265	total: 539ms	remaining: 3.95s
12:	learn: 0.4728835	total: 578ms	remaining: 3.87s
13:	learn: 0.4688841	total: 618ms	remaining: 3.79s
14:	learn: 0.4652408	total: 660ms	remaining: 3.74s
15:	learn: 0.4629365	total: 702ms	remaining: 3.69s
16:	learn: 0.4585292	total: 744ms	remaining: 3.63s
17:	learn: 0.4560397	total: 784ms	remaining: 3.57s
18:	learn: 0.4541882	total: 825ms	remaining: 3.52s
19:	learn:

In [0]:
idd=ntest["id"]
ntest.drop(['id'], axis=1, inplace=True)
final_status = xgb_gscv.predict_proba(ntest)
ntest["id"]=idd
test["redemption_status"]=final_status[:,1]
test.to_csv('model_output.csv',columns=['id','redemption_status'])

# R

In [0]:
# activate R magic
%load_ext rpy2.ipython

In [0]:
%%R
#install.packages("ROSE")
#install.packages("caret")
#install.packages("e1071")
install.packages("pROC")

In [0]:
%%R
rm(list=ls())
train=read.csv("/content/drive/My Drive/My Colab/AmExpert2019/train.csv",header = T)
test=read.csv("/content/drive/My Drive/My Colab/AmExpert2019/test.csv",header = T)
table(train$redemption_status)
dim(train)

idd=test$id
train <- subset(train, select = -c(X))
test <- subset(test, select = -c(id))

cols.num <- colnames(train)
train[cols.num] <- sapply(train[cols.num],as.numeric)
cols.num <- colnames(test)
test[cols.num] <- sapply(test[cols.num],as.numeric)
###################################################
library(caret)
set.seed(5)
index <- createDataPartition(train$redemption_status, p=0.80, list=FALSE) #data partition
ntrain <- train[ index,]       #Traning data=75%
ntest<- train[-index,]         #Test data=25%

#######################################################
library(ROSE)
#data <- ROSE(redemption_status ~ ., data = ntrain, seed = 1)$data
#table(data$redemption_status)
data.sample <- ovun.sample(redemption_status ~ ., data =ntrain, method = "over",N = 124286)$data
table(data.sample$redemption_status)
#########################################################
#DT
library(rpart)
ntrain$redemption_status=as.factor(ntrain$redemption_status)  # converting response variable in factor
data.sample$redemption_status <- factor(data.sample$redemption_status)
tree.rose <- rpart(redemption_status ~ ., data = data.sample)
pred.tree.rose <- predict(tree.rose, newdata = ntest)
for_auc=predict(tree.rose,ntest,type="prob")
library(pROC)
a=auc(ntest$redemption_status,for_auc[,2])
a
#RF
install.packages("randomForest")
library(randomForest)
ntrain$redemption_status=as.factor(ntrain$redemption_status)  # converting response variable in factor
data.sample$redemption_status <- factor(data.sample$redemption_status)
rf.rose <- randomForest(redemption_status ~ ., data = data.sample)
pred.rf.rose <- predict(rf.rose, newdata = ntest)
for_auc=predict(rf.rose,ntest,type="prob")
library(pROC)
a=auc(ntest$redemption_status,for_auc[,2])
a

Area under the curve: 0.6926


In [0]:
%%R
pred=predict(rf.rose,test,type = "class")
submit <- data.frame(id= idd, redemption_status =pred)
#submit$redemption_status=ifelse(submit$redemption_status=='0.246354049809289',1,0)
write.csv(submit, file = "result.csv", row.names = FALSE)


## H2O + R

In [0]:
%%R
install.packages("h2o")
library(h2o)
h2o.init()

/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: also installing the dependencies ‘bitops’, ‘RCurl’


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'https://cran.rstudio.com/src/contrib/bitops_1.0-6.tar.gz'

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 8734 bytes

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: downloaded 8734 bytes


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'https://cran.rstudio.com/src/contrib/RCurl_1.95-4.12.tar.gz'

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 916763 bytes (895 KB)

  warnings.wa


H2O is not running yet, starting it now...

Note:  In case of errors look at the following log files:
    /tmp/RtmpUcMCqO/h2o_UnknownUser_started_from_r.out
    /tmp/RtmpUcMCqO/h2o_UnknownUser_started_from_r.err


Starting H2O JVM and connecting: .. Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         2 seconds 637 milliseconds 
    H2O cluster timezone:       Etc/UTC 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.26.0.2 
    H2O cluster version age:    2 months and 5 days  
    H2O cluster name:           H2O_started_from_R_root_vgj064 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   6.25 GB 
    H2O cluster total cores:    4 
    H2O cluster allowed cores:  4 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    H2O API Extensions:         Amazon S3, XGBoost

In [0]:
%%R
rm(list=ls())
train=read.csv("/content/drive/My Drive/My Colab/AmExpert2019/train.csv",header = T)
test=read.csv("/content/drive/My Drive/My Colab/AmExpert2019/test.csv",header = T)
table(train$redemption_status)
dim(train)

idd=test$id
train <- subset(train, select = -c(X))
test <- subset(test, select = -c(id))

cols.num <- colnames(train)
train[cols.num] <- sapply(train[cols.num],as.numeric)
cols.num <- colnames(test)
test[cols.num] <- sapply(test[cols.num],as.numeric)
###################################################
library(caret)
set.seed(5)
index <- createDataPartition(train$redemption_status, p=0.80, list=FALSE) #data partition
ntrain <- train[ index,]       #Traning data=75%
ntest<- train[-index,]         #Test data=25%

#######################################################
library(ROSE)
data.sample <- ovun.sample(redemption_status ~ ., data =ntrain, method = "over",N = 124286)$data
table(data.sample$redemption_status)

/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Loading required package: lattice

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Loading required package: ggplot2

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Loaded ROSE 0.0-3


  warnings.warn(x, RRuntimeWarning)



    0     1 
62107 62179 


In [0]:
%%R
data.sample=as.h2o(data.sample)
ntest=as.h2o(ntest)
test=as.h2o(test)

  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%


In [0]:
%%R
# Identify predictors and response
y <- "redemption_status"
x <- setdiff(names(data.sample), y)

# For binary classification, response should be a factor
data.sample[,y] <- as.factor(data.sample[,y])
ntest[,y] <- as.factor(ntest[,y])


In [0]:
%%R
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml <- h2o.automl(x = x, y = y,
                  training_frame = data.sample,
                  max_models = 20,
                  seed = 1)

# View the AutoML Leaderboard
lb <- aml@leaderboard

  |======================================================================| 100%


In [0]:
%%R
lb

                                             model_id       auc    logloss
1 StackedEnsemble_BestOfFamily_AutoML_20191002_150304 0.9984764 0.03489959
2                    XGBoost_1_AutoML_20191002_150304 0.9984757 0.07720477
3    StackedEnsemble_AllModels_AutoML_20191002_150304 0.9983649 0.03215518
4                    XGBoost_2_AutoML_20191002_150304 0.9969771 0.12453055
  mean_per_class_error       rmse         mse
1          0.004266028 0.08763605 0.007680077
2          0.004258183 0.12300027 0.015129067
3          0.004660704 0.08403435 0.007061771
4          0.008098926 0.16087046 0.025879304

[4 rows x 6 columns] 


In [0]:
%%R
pred <- h2o.predict(aml@leader, test)

  |======================================================================| 100%


In [0]:
%%R
pred

  predict        p0           p1
1       0 0.9998538 1.462060e-04
2       0 0.9999331 6.689913e-05
3       0 0.9999231 7.686060e-05
4       0 0.9999355 6.446786e-05
5       0 0.9999328 6.716931e-05
6       0 0.9999360 6.400892e-05

[50226 rows x 3 columns] 


In [0]:
%%R
pred=as.data.frame(pred$predict)
result=cbind(idd,pred)
head(result)

  idd predict
1   3       0
2   4       0
3   5       0
4   8       0
5  10       0
6  12       0


In [0]:
%%R
write.csv(result, file = "result.csv", row.names = FALSE)

## H2O

In [0]:
pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
     |████████████████████████████████| 123.6MB 1.1MB/s 
  Created wheel for h2o: filename=h2o-3.26.0.6-py2.py3-none-any.whl size=123690782 sha256=c9476653e1f58ec77976a349eb8d7d212145062ff3734eb40faf9c4999d4e663
  Stored in directory: /root/.cache/pip/wheels/eb/1a/72/252857fc093eaad70a7f91ddbcabf1dafd38fb5e6c02bcf9b3
Successfully built h2o


In [0]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from keras.utils import np_utils
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import h2o
import math
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator
from h2o.grid.grid_search import H2OGridSearch
from sklearn.model_selection import train_test_split
from h2o.automl import H2OAutoML
h2o.init(nthreads=-1, strict_version_check=True)

Using TensorFlow backend.


Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.4" 2019-07-16; OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3); OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpou4i8vjj
  JVM stdout: /tmp/tmpou4i8vjj/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpou4i8vjj/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.6
H2O cluster version age:,3 days
H2O cluster name:,H2O_from_python_unknownUser_780bgt
H2O cluster total nodes:,1
H2O cluster free memory:,6.250 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [0]:
train=pd.concat([X_train,y_train],axis=1)
test=pd.concat([X_test,y_test],axis=1)

train=h2o.H2OFrame(train)
test=h2o.H2OFrame(test)

x = train.columns
y = "redemption_status"

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
mod = H2OAutoML(max_models=20, seed=1)
mod.train(x=x, y=y, training_frame=train)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%


In [0]:
mod.leaderboard

model_id,auc,logloss,mean_per_class_error,rmse,mse
XGBoost_1_AutoML_20191005_093738,0.998859,0.0665048,0.00316549,0.113441,0.0128687
StackedEnsemble_BestOfFamily_AutoML_20191005_093738,0.998857,0.0311339,0.00315629,0.082164,0.00675092
StackedEnsemble_AllModels_AutoML_20191005_093738,0.998797,0.028141,0.00345995,0.0781357,0.00610518
XGBoost_2_AutoML_20191005_093738,0.997847,0.106019,0.00611933,0.146556,0.0214788
XGBoost_3_AutoML_20191005_093738,0.848454,0.512417,0.237255,0.410912,0.168848


In [0]:
#ntest=h2o.H2OFrame(ntest)
a=mod.leader.model_performance(test)
a


ModelMetricsBinomial: xgboost
** Reported on test data. **

MSE: 0.022292672838654076
RMSE: 0.14930731006435713
LogLoss: 0.08306043176338451
Mean Per-Class Error: 0.17288721970974685
AUC: 0.8972649467404986
pr_auc: 0.13577043097193553
Gini: 0.7945298934809972

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6837389213698251: 


,,0,1,Error,Rate
0,0,23037.0,267.0,0.0115,(267.0/23304.0)
1,1,132.0,75.0,0.6377,(132.0/207.0)
2,Total,23169.0,342.0,0.017,(399.0/23511.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.683739,0.273224,66.0
1,max f2,0.609836,0.334110,85.0
2,max f0point5,0.779124,0.242009,42.0
3,max accuracy,0.992463,0.991153,0.0
4,max precision,0.960324,0.321429,6.0
5,max recall,0.000086,1.000000,399.0
6,max specificity,0.992463,0.999957,0.0
7,max absolute_mcc,0.683739,0.273760,66.0
8,max min_per_class_accuracy,0.057003,0.821256,314.0
9,max mean_per_class_accuracy,0.082121,0.827113,294.0



Gains/Lift Table: Avg response rate:  0.88 %, avg score:  5.26 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.010038,0.765554,25.988578,25.988578,0.228814,0.872373,0.228814,0.872373,0.260870,0.260870,2498.857775,2498.857775
1,,2,0.020033,0.600684,15.466173,20.738546,0.136170,0.676118,0.182590,0.774454,0.154589,0.415459,1446.617330,1973.854580
2,,3,0.030028,0.476512,4.833179,15.444267,0.042553,0.534340,0.135977,0.694529,0.048309,0.463768,383.317916,1444.426654
3,,4,0.040024,0.379864,7.733087,13.518520,0.068085,0.425617,0.119022,0.627373,0.077295,0.541063,673.308665,1251.852023
4,,5,0.050019,0.313270,2.899907,11.396604,0.025532,0.346525,0.100340,0.571251,0.028986,0.570048,189.990749,1039.660357
5,,6,0.100038,0.142985,2.800860,7.098732,0.024660,0.211235,0.062500,0.391243,0.140097,0.710145,180.086020,609.873188
6,,7,0.150015,0.076857,1.643281,5.281279,0.014468,0.104504,0.046498,0.295718,0.082126,0.792271,64.328091,428.127941
7,,8,0.200077,0.046595,0.868494,4.177145,0.007647,0.060374,0.036777,0.236832,0.043478,0.835749,-13.150604,317.714495
8,,9,0.300030,0.018511,0.434986,2.930466,0.003830,0.029976,0.025801,0.167919,0.043478,0.879227,-56.501388,193.046601
9,,10,0.400026,0.007968,0.338179,2.282463,0.002977,0.012401,0.020096,0.129043,0.033816,0.913043,-66.182136,128.246307


In [0]:
#ntest=ntest.as_data_frame()
idd=ntest["id"]
ntest.drop(['id'], axis=1, inplace=True)

In [0]:
ntest=h2o.H2OFrame(ntest)
pre=mod.leader.predict(ntest)
pre.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%
xgboost prediction progress: |████████████████████████████████████████████| 100%


predict,p0,p1
0,0.986409,0.0135907
0,0.997442,0.00255753
0,0.998522,0.00147765
0,0.999848,0.000151975
0,0.9969,0.00309967
0,0.999953,4.68658e-05
0,0.996645,0.00335475
0,0.999635,0.000364935
0,0.999929,7.11816e-05
0,0.99887,0.00112986


In [0]:
x=pre.as_data_frame()
x.to_csv("h2o_over_result.csv")

In [0]:
x=pre["predict"].as_data_frame()
x["predict"].value_counts()

0    50222
1        4
Name: predict, dtype: int64

In [0]:
pred=pd.read_csv("/content/drive/My Drive/My Colab/AmExpert2019/sample_submission_Byiv0dS.csv")
pred.head()

,id,redemption_status
0,3,0
1,4,0
2,5,0
3,8,0
4,10,0


In [0]:
pred["redemption_status"]=x["predict"]
pred.head()

,id,redemption_status
0,3,0
1,4,0
2,5,0
3,8,0
4,10,0


In [0]:
pred.to_csv("result.csv")

## Model_22

In [0]:
pip install catboost

In [0]:
# Import dependencies
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error,confusion_matrix, precision_score, recall_score, auc,roc_curve
from sklearn import ensemble, linear_model, neighbors, svm, tree, neural_network
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn import svm,model_selection, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
import matplotlib.pyplot as plt
from math import sqrt
import seaborn as sns
import warnings
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
warnings.filterwarnings('ignore')

In [0]:
#https://www.kaggle.com/aldemuro/comparing-ml-algorithms-train-accuracy-90
MLA = [
    #Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    #Gaussian Processes
    gaussian_process.GaussianProcessClassifier(),
    
    #GLM
    linear_model.LogisticRegressionCV(),
    linear_model.PassiveAggressiveClassifier(),
    linear_model. RidgeClassifierCV(),
    linear_model.SGDClassifier(),
    linear_model.Perceptron(),
    
    #Navies Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),
    
    #Nearest Neighbor
    neighbors.KNeighborsClassifier(),
    
    #SVM
    svm.SVC(probability=True),
    svm.NuSVC(probability=True),
    svm.LinearSVC(),
    
    #Trees    
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),
    
    #others
    XGBClassifier(),              
    LGBMClassifier(),             
    CatBoostClassifier(),          
    MLPClassifier()              
    ]

MLA_columns = []
MLA_compare = pd.DataFrame(columns = MLA_columns)


row_index = 0
for alg in MLA:
    
    
    predicted = alg.fit(X_train, y_train).predict(X_test)
    fp, tp, th = roc_curve(y_test, predicted)
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index,'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Train Accuracy'] = round(alg.score(X_train, y_train), 4)
    MLA_compare.loc[row_index, 'MLA Test Accuracy'] = round(alg.score(X_test, y_test), 4)
    MLA_compare.loc[row_index, 'MLA Precission'] = precision_score(y_test, predicted)
    MLA_compare.loc[row_index, 'MLA Recall'] = recall_score(y_test, predicted)
    MLA_compare.loc[row_index, 'MLA AUC'] = auc(fp, tp)

    row_index+=1
    
MLA_compare.sort_values(by = ['MLA AUC'], ascending = False, inplace = True)


In [0]:
MLA_compare

NameError: ignored